## 1. Preparing our dataset
<p><em>These recommendations are so on point! How does this playlist know me so well?</em></p>
<p><img src="https://assets.datacamp.com/production/project_449/img/iphone_music.jpg" alt="Project Image Record" width="600px"></p>
<p>Over the past few years, streaming services with huge catalogs have become the primary means through which most people listen to their favorite music. But at the same time, the sheer amount of music on offer can mean users might be a bit overwhelmed when trying to look for newer music that suits their tastes.</p>
<p>For this reason, streaming services have looked into means of categorizing music to allow for personalized recommendations. One method involves direct analysis of the raw audio information in a given song, scoring the raw data on a variety of metrics. Today, we'll be examining data compiled by a research group known as The Echo Nest. Our goal is to look through this dataset and classify songs as being either 'Hip-Hop' or 'Rock' - all without listening to a single one ourselves. In doing so, we will learn how to clean our data, do some exploratory data visualization, and use feature reduction towards the goal of feeding our data through some simple machine learning algorithms, such as decision trees and logistic regression.</p>
<p>To begin with, let's load the metadata about our tracks alongside the track metrics compiled by The Echo Nest. A song is about more than its title, artist, and number of listens. We have another dataset that has musical features of each track such as <code>danceability</code> and <code>acousticness</code> on a scale from -1 to 1. These exist in two different files, which are in different formats - CSV and JSON. While CSV is a popular file format for denoting tabular data, JSON is another common file format in which databases often return the results of a given query.</p>
<p>Let's start by creating two pandas <code>DataFrames</code> out of these files that we can merge so we have features and labels (often also referred to as <code>X</code> and <code>y</code>) for the classification later on.</p>

In [10]:
import pandas as pd

# Read in track metadata with genre labels
tracks = pd.read_csv('datasets/fma-rock-vs-hiphop.csv')

# Read in track metrics with the features
echonest_metrics = pd.read_json('datasets/echonest-metrics.json')

# Merge the relevant columns of tracks and echonest_metrics
echo_tracks = echonest_metrics.merge(tracks[['track_id', 'genre_top']], on='track_id')

# Inspect the resultant dataframe
echo_tracks

,acousticness,danceability,energy,instrumentalness,liveness,speechiness,tempo,track_id,valence,genre_top
0,0.416675,0.675894,0.634476,1.062807e-02,0.177647,0.159310,165.922,2,0.576661,Hip-Hop
1,0.374408,0.528643,0.817461,1.851103e-03,0.105880,0.461818,126.957,3,0.269240,Hip-Hop
2,0.977282,0.468808,0.134975,6.876998e-01,0.105381,0.073124,119.646,341,0.430707,Rock
3,0.953349,0.498525,0.552503,9.243910e-01,0.684914,0.028885,78.958,46204,0.430448,Rock
4,0.613229,0.500320,0.487992,9.368113e-01,0.637750,0.030327,112.667,46205,0.824749,Rock
5,0.974939,0.273629,0.447933,9.610854e-01,0.091992,0.039337,81.639,46206,0.703647,Rock
6,0.315471,0.239752,0.946471,7.763353e-01,0.703158,0.091139,117.667,46262,0.077510,Rock
7,0.013804,0.097749,0.804726,7.819637e-01,0.262923,0.041303,181.104,46263,0.243307,Rock
8,0.080884,0.292023,0.935993,5.550325e-01,0.726872,0.067671,119.512,46264,0.054772,Rock
9,0.352181,0.297451,0.881247,1.143533e-01,0.718530,0.081976,122.257,46265,0.066342,Rock


In [11]:
echonest_metrics

,acousticness,danceability,energy,instrumentalness,liveness,speechiness,tempo,track_id,valence
0,0.416675,0.675894,0.634476,1.062807e-02,0.177647,0.159310,165.922,2,0.576661
1,0.374408,0.528643,0.817461,1.851103e-03,0.105880,0.461818,126.957,3,0.269240
10,0.235506,0.438672,0.487752,7.161218e-01,0.070359,0.047298,120.790,145,0.650452
100,0.977282,0.468808,0.134975,6.876998e-01,0.105381,0.073124,119.646,341,0.430707
1000,0.081256,0.897197,0.367886,1.044740e-01,0.085845,0.041271,100.744,3457,0.749293
10000,0.069876,0.403267,0.442406,8.394255e-01,0.094897,0.058049,185.527,46156,0.053260
10001,0.970244,0.298115,0.459427,9.830320e-01,0.097634,0.300604,238.737,46157,0.025365
10002,0.946941,0.124713,0.067169,9.562725e-01,0.102589,0.038882,92.585,46158,0.035686
10003,0.915842,0.225577,0.588263,8.726189e-01,0.110507,0.068540,70.967,46159,0.180629
10004,0.292440,0.724268,0.569141,3.041421e-01,0.096222,0.079397,163.989,46160,0.308078


## 2. Pairwise relationships between continuous variables
<p>We typically want to avoid using variables that have strong correlations with each other -- hence avoiding feature redundancy -- for a few reasons:</p>
<ul>
<li>To keep the model simple and improve interpretability (with many features, we run the risk of overfitting).</li>
<li>When our datasets are very large, using fewer features can drastically speed up our computation time.</li>
</ul>
<p>To get a sense of whether there are any strongly correlated features in our data, we will use built-in functions in the <code>pandas</code> package.</p>

In [13]:
# Create a correlation matrix
corr_metrics = ...
corr_metrics.style.background_gradient()

AttributeError: 'ellipsis' object has no attribute 'style'

## 3. Normalizing the feature data
<p>As mentioned earlier, it can be particularly useful to simplify our models and use as few features as necessary to achieve the best result. Since we didn't find any particular strong correlations between our features, we can instead use a common approach to reduce the number of features called <strong>principal component analysis (PCA)</strong>. </p>
<p>It is possible that the variance between genres can be explained by just a few features in the dataset. PCA rotates the data along the axis of highest variance, thus allowing us to determine the relative contribution of each feature of our data towards the variance between classes. </p>
<p>However, since PCA uses the absolute variance of a feature to rotate the data, a feature with a broader range of values will overpower and bias the algorithm relative to the other features. To avoid this, we must first normalize our data. There are a few methods to do this, but a common way is through <em>standardization</em>, such that all features have a mean = 0 and standard deviation = 1 (the resultant is a z-score).</p>

In [ ]:
# Define our features 
features = ...

# Define our labels
labels = ...

# Import the StandardScaler
...

# Scale the features and set the values to a new variable
scaler = ...
scaled_train_features = ...

## 4. Principal Component Analysis on our scaled data
<p>Now that we have preprocessed our data, we are ready to use PCA to determine by how much we can reduce the dimensionality of our data. We can use <strong>scree-plots</strong> and <strong>cumulative explained ratio plots</strong> to find the number of components to use in further analyses.</p>
<p>Scree-plots display the number of components against the variance explained by each component, sorted in descending order of variance. Scree-plots help us get a better sense of which components explain a sufficient amount of variance in our data. When using scree plots, an 'elbow' (a steep drop from one data point to the next) in the plot is typically used to decide on an appropriate cutoff.</p>

In [ ]:
# This is just to make plots appear in the notebook
%matplotlib inline

# Import our plotting module, and PCA class
#... YOUR CODE ...

# Get our explained variance ratios from PCA using all features
pca = ...
...
exp_variance = ...

# plot the explained variance using a barplot
fig, ax = plt.subplots()
ax.bar(..., ...)
ax.set_xlabel('Principal Component #')

## 5. Further visualization of PCA
<p>Unfortunately, there does not appear to be a clear elbow in this scree plot, which means it is not straightforward to find the number of intrinsic dimensions using this method. </p>
<p>But all is not lost! Instead, we can also look at the <strong>cumulative explained variance plot</strong> to determine how many features are required to explain, say, about 85% of the variance (cutoffs are somewhat arbitrary here, and usually decided upon by 'rules of thumb'). Once we determine the appropriate number of components, we can perform PCA with that many components, ideally reducing the dimensionality of our data.</p>

In [ ]:
# Import numpy
...

# Calculate the cumulative explained variance
cum_exp_variance = ...

# Plot the cumulative explained variance and draw a dashed line at 0.85.
fig, ax = plt.subplots()
...
ax.axhline(y=0.85, linestyle='--')

# choose the n_components where about 85% of our variance can be explained
n_components = ...

# Perform PCA with the chosen number of components and project data onto components
pca = PCA(n_components, random_state=10)
pca.fit(scaled_train_features)
pca_projection = ...

## 6. Train a decision tree to classify genre
<p>Now we can use the lower dimensional PCA projection of the data to classify songs into genres. To do that, we first need to split our dataset into 'train' and 'test' subsets, where the 'train' subset will be used to train our model while the 'test' dataset allows for model performance validation.</p>
<p>Here, we will be using a simple algorithm known as a decision tree. Decision trees are rule-based classifiers that take in features and follow a 'tree structure' of binary decisions to ultimately classify a data point into one of two or more categories. In addition to being easy to both use and interpret, decision trees allow us to visualize the 'logic flowchart' that the model generates from the training data.</p>
<p>Here is an example of a decision tree that demonstrates the process by which an input image (in this case, of a shape) might be classified based on the number of sides it has and whether it is rotated.</p>
<p><img src="https://assets.datacamp.com/production/project_449/img/simple_decision_tree.png" alt="Decision Tree Flow Chart Example" width="350px"></p>

In [ ]:
# Import train_test_split function and Decision tree classifier
# ... YOUR CODE ...

# Split our data
train_features, test_features, train_labels, test_labels = ...

# Train our decision tree
tree = ...
...

# Predict the labels for the test data
pred_labels_tree = ...

## 7. Compare our decision tree to a logistic regression
<p>Although our tree's performance is decent, it's a bad idea to immediately assume that it's therefore the perfect tool for this job -- there's always the possibility of other models that will perform even better! It's always a worthwhile idea to at least test a few other algorithms and find the one that's best for our data.</p>
<p>Sometimes simplest is best, and so we will start by applying <strong>logistic regression</strong>. Logistic regression makes use of what's called the logistic function to calculate the odds that a given data point belongs to a given class. Once we have both models, we can compare them on a few performance metrics, such as false positive and false negative rate (or how many points are inaccurately classified). </p>

In [ ]:
# Import LogisticRegression
from sklearn.linear_model import LogisticRegression

# Train our logistic regression and predict labels for the test set
logreg = ...
...
pred_labels_logit = ...

# Create the classification report for both models
from sklearn.metrics import classification_report
class_rep_tree = ...
class_rep_log = ...

print("Decision Tree: \n", class_rep_tree)
print("Logistic Regression: \n", class_rep_log)

## 8. Balance our data for greater performance
<p>Both our models do similarly well, boasting an average precision of 87% each. However, looking at our classification report, we can see that rock songs are fairly well classified, but hip-hop songs are disproportionately misclassified as rock songs. </p>
<p>Why might this be the case? Well, just by looking at the number of data points we have for each class, we see that we have far more data points for the rock classification than for hip-hop, potentially skewing our model's ability to distinguish between classes. This also tells us that most of our model's accuracy is driven by its ability to classify just rock songs, which is less than ideal.</p>
<p>To account for this, we can weight the value of a correct classification in each class inversely to the occurrence of data points for each class. Since a correct classification for "Rock" is not more important than a correct classification for "Hip-Hop" (and vice versa), we only need to account for differences in <em>sample size</em> of our data points when weighting our classes here, and not relative importance of each class. </p>

In [ ]:
# Subset only the hip-hop tracks, and then only the rock tracks
hop_only = ...
rock_only = ...

# sample the rocks songs to be the same number as there are hip-hop songs
rock_only = ...

# concatenate the dataframes rock_only and hop_only
rock_hop_bal = ...

# The features, labels, and pca projection are created for the balanced dataframe
features = rock_hop_bal.drop(['genre_top', 'track_id'], axis=1) 
labels = rock_hop_bal['genre_top']
pca_projection = pca.fit_transform(scaler.fit_transform(features))

# Redefine the train and test set with the pca_projection from the balanced data
train_features, test_features, train_labels, test_labels = train_test_split(..., random_state=10)

## 9. Does balancing our dataset improve model bias?
<p>We've now balanced our dataset, but in doing so, we've removed a lot of data points that might have been crucial to training our models. Let's test to see if balancing our data improves model bias towards the "Rock" classification while retaining overall classification performance. </p>
<p>Note that we have already reduced the size of our dataset and will go forward without applying any dimensionality reduction. In practice, we would consider dimensionality reduction more rigorously when dealing with vastly large datasets and when computation times become prohibitively large.</p>

In [ ]:
# Train our decision tree on the balanced data
tree = ...
...
pred_labels_tree = ...

# Train our logistic regression on the balanced data
logreg = ...
...
pred_labels_logit = ...

# Compare the models
print("Decision Tree: \n", classification_report(...))
print("Logistic Regression: \n", classification_report(...))

## 10. Using cross-validation to evaluate our models
<p>Success! Balancing our data has removed bias towards the more prevalent class. To get a good sense of how well our models are actually performing, we can apply what's called <strong>cross-validation</strong> (CV). This step allows us to compare models in a more rigorous fashion.</p>
<p>Since the way our data is split into train and test sets can impact model performance, CV attempts to split the data multiple ways and test the model on each of the splits. Although there are many different CV methods, all with their own advantages and disadvantages, we will use what's known as <strong>K-fold</strong> CV here. K-fold first splits the data into K different, equally sized subsets. Then, it iteratively uses each subset as a test set while using the remainder of the data as train sets. Finally, we can then aggregate the results from each fold for a final model performance score.</p>

In [ ]:
from sklearn.model_selection import KFold, cross_val_score

# Set up our K-fold cross-validation
kf = ...

tree = DecisionTreeClassifier(random_state=10)
logreg = LogisticRegression(random_state=10)

# Train our models using KFold cv
tree_score = ...
logit_score = ...

# Print the mean of each array of scores
print("Decision Tree:", ..., "Logistic Regression:", ...)